In [1]:
from keras.models import Sequential, load_model
import pandas as pd
import numpy as np
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import random
import datetime

from Indicator import Indicator

def check_trend(df):
    df.loc[(df['Close'] > df['MA']) &
    ((df['Close'] > df['Close'].shift(1)) | (df['MA'] > df['MA'].shift(1))) &
    ((df['Close'] > df['Close'].shift(2)) | (df['MA'] > df['MA'].shift(2))) &
    ((df['Close'] > df['Close'].shift(3)) | (df['MA'] > df['MA'].shift(3))) &
    ((df['Close'] > df['Close'].shift(4)) | (df['MA'] > df['MA'].shift(4))) &
    ((df['Close'] > df['Close'].shift(5)) | (df['MA'] > df['MA'].shift(5))) , 'Trend'] = 'Up'

    df.loc[(df['Close'] < df['MA']) &
    ((df['Close'] < df['Close'].shift(1)) | (df['MA'] < df['MA'].shift(1))) &
    ((df['Close'] < df['Close'].shift(2)) | (df['MA'] < df['MA'].shift(2))) &
    ((df['Close'] < df['Close'].shift(3)) | (df['MA'] < df['MA'].shift(3))) &
    ((df['Close'] < df['Close'].shift(4)) | (df['MA'] < df['MA'].shift(4))) &
    ((df['Close'] < df['Close'].shift(5)) | (df['MA'] < df['MA'].shift(5))) , 'Trend'] = 'Down'
    
    df.loc[df['Trend'].isnull(), 'Ntrend'] = df['Trend'].shift(1)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull(),'Ntrend'] = df['Trend'].shift(2)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull(),'Ntrend'] = df['Trend'].shift(3)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull(),'Ntrend'] = df['Trend'].shift(4)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull() & df['Trend'].shift(4).isnull(),'Ntrend'] = df['Trend'].shift(5)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull() & df['Trend'].shift(4).isnull() & df['Trend'].shift(5).isnull(),'Ntrend'] = df['Trend'].shift(6)
    
    df['S1'] = df['Close'].shift(-1)
    df['S2'] = df['Close'].shift(-2)
    df.loc[df['S1'].isnull(),'S1'] = 0
    df.loc[df['S2'].isnull(),'S2'] = 0

   
    #Trading Signal (0,1)
    df.loc[(df['Trend'] == 'Up') | (df['Ntrend'] == 'Up'), 'Trading Signal'] = ((abs(df['Close'] - df[['Close','S1','S2']].min(axis = 1)) / 
    abs(df[['Close','S1','S2']].max(axis = 1) - df[['Close','S1','S2']].min(axis = 1)))*0.5) + 0.5
    df.loc[(df['Trend'] == 'Down') | (df['Ntrend'] == 'Down'), 'Trading Signal'] = ((abs(df['Close'] - df[['Close','S1','S2']].min(axis = 1)) / 
    abs(df[['Close','S1','S2']].max(axis = 1) - df[['Close','S1','S2']].min(axis = 1)))*0.5)
   
    df.loc[df['Trend'].isnull(),'Trend'] =df['Ntrend']
    
    df = df.drop(['Ntrend','S1','S2'],axis=1)
        
    return df

def fillindicator(df):
    indicator = Indicator()
    train_set = indicator.MA(df,15)
    train_set = indicator.MACD(df)
    train_set = indicator.STOCHASTIC(df)
    train_set = indicator.RSI(df)
    train_set = indicator.WILLIAMSR(df)
    train_set = indicator.DMI(df)
    train_set = indicator.AROON(df)
    train_set = indicator.ATR(df)
    train_set = indicator.CCI(df)
    train_set = indicator.OBV(df)
    return train_set

def norm(df,col_name):# this will get columnname + _n
    #Normalize data each colunm in to range -1 to 1
    df[col_name+'_n'] = (df[col_name]-df[col_name].min())/(df[col_name].max()-df[col_name].min())
    return df

def signal_to_num(df):
    df.loc[df['Trend']=='Down','Trend'] = 0
    df.loc[df['Trend']=='Up','Trend'] = 1
    return df

In [86]:
start = datetime.datetime(2019,1,1)#26 day before start year for make indicator
end = datetime.datetime(2021,1,1)

money = 100000
stocklist = ['AES','EBAY','VRTX','GOOGL','IP','CSCO','UAL','NVR','TPR','BA','KTC.bk']
# stock = 'KTC.bk'

In [87]:
#Hybrid Stock Model
HBstock = []
for stock in stocklist:
    df = web.DataReader(stock,'yahoo',start,end)
    df = fillindicator(df)
    df = norm(df,'MA')
    df = norm(df,'MACD')
    df = norm(df,'SIGNAL LINE')
    df = norm(df,'%K')
    df = norm(df,'%D')
    df = norm(df,'%R')
    df = norm(df,'RSI-7')
    df = norm(df,'RSI-14')
    df = check_trend(df)
    df = df.iloc[61:]
    df = signal_to_num(df)

    model = load_model('Model/best_4.h5')

    test_x = df[['MA_n','MACD_n','SIGNAL LINE_n','RSI-7_n','%K_n','%D_n','%R_n','Trading Signal','Trend']]
    x_test = test_x[['MA_n','MACD_n','SIGNAL LINE_n','RSI-7_n','%K_n','%D_n','%R_n','Trading Signal']].values
    y_test = test_x['Trend'].values
    x_test=np.asarray(x_test).astype(np.float32)
    y_test=np.asarray(y_test).astype(np.float32)

    pred = model.predict(x_test)

    data = df
    data['pred'] = pred
    mean = data['pred'].mean()
    data['pred'] =np.where(data['pred'] >= mean,'up','down')

    data['up'] = np.where(data['pred']=='up',data['Close'],None)
    data['down'] = np.where(data['pred']=='down',data['Close'],None)

    b = []
    s = []
    flag = 'sell'
    for i,r in data.iterrows():
        if (flag=='sell') & (r['pred']=='up'):
            b.append(r['Close'])
            flag = 'buy'
        elif (flag=='buy') & (r['pred']=='down'):
            s.append(r['Close'])
            flag = 'sell'

    if len(b)>len(s):
        del b[-1]

    modelbuy = len(b)
    modelsell = len(s)
    pfmodel = sum((np.array(s)*money)-(np.array(b)*money))
    profitmodel = []
    for i in range(len(b)):
        profitmodel.append((s[i]*money-b[i]*money)/(b[i]*money*100))
    profitmodel = sum(profitmodel)
    HBstock.append([stock,profitmodel,modelbuy,modelsell,pfmodel])

In [88]:
#MACD Simulation
MACD=[]
for stock in stocklist:
    df = web.DataReader(stock,'yahoo',start,end)
    df = fillindicator(df)
    df = df.iloc[61:]

    df['signal'] = 0
    df.loc[((df['MACD']>df['SIGNAL LINE'])&(df['MACD'].shift()<df['SIGNAL LINE'].shift())),'signal'] = 1
    df.loc[((df['MACD']<df['SIGNAL LINE'])&(df['MACD'].shift()>df['SIGNAL LINE'].shift())),'signal'] = 2

    prd = []
    for i ,row in df.iterrows():
        if row['signal']==1:
    #         plt.scatter(x=i, y=row['Close'], c='green')
            prd.append(('buy',row['Close']))
        elif row['signal']==2:
    #         plt.scatter(x=i, y=row['Close'], c='red')
            prd.append(('sell',row['Close']))

    if prd[0][0]=='sell':
        del prd[0]
    if len(prd) %2!=0:
        del prd[-1]

    b = []
    s = []
    for i in prd:
        if i[0] =='buy':
            b.append(i[1])
        else:
            s.append(i[1])
    macdbuy = len(b)
    macdsell = len(s)
    pfmacd=sum(np.array(s)*money-np.array(b)*money)
    profitmacd = []
    for i in range(len(b)):
        profitmacd.append((s[i]*money-b[i]*money)/(b[i]*money*100))
    profitmacd = sum(profitmacd)
    MACD.append([stock,profitmacd,macdbuy,macdsell,pfmacd])

In [89]:
print('MACD')
print('Profit % = ',profitmacd,'\n','buycount = ',macdbuy,'sellcount =',macdsell,'\n','Profit',pfmacd)
print('---------------------------------------------------------------------------------------------------')
print('Hybrid Stock Algorithm')
print('Profit % = ',profitmodel,'\n','buycount = ',modelbuy,'sellcount =',modelsell,'\n','Profit',pfmodel)

MACD
Profit % =  0.007924402337792848 
 buycount =  13 sellcount = 13 
 Profit 2375000.0
---------------------------------------------------------------------------------------------------
Hybrid Stock Algorithm
Profit % =  0.006833473457279245 
 buycount =  26 sellcount = 26 
 Profit 2400000.0


In [90]:
for i in range(len(MACD)):
    print(HBstock[i])
    print(MACD[i])
    print('-----------------')

['AES', 0.0006663080116829938, 29, 29, 141999.9122619629]
['AES', 0.002289241266495786, 16, 16, 386999.9885559082]
-----------------
['EBAY', -0.0002661597920504183, 31, 31, -671999.1683959961]
['EBAY', 0.003145573477452437, 19, 19, 967000.0076293945]
-----------------
['VRTX', -0.006662227526845908, 43, 43, -18319993.591308594]
['VRTX', 0.000336487877994895, 14, 14, -2770999.1455078125]
-----------------
['GOOGL', -0.000409470407995956, 35, 35, -7708947.75390625]
['GOOGL', 0.00491381899257385, 14, 14, 62563952.63671875]
-----------------
['IP', -0.003494178387799569, 33, 33, -1352999.8779296875]
['IP', 0.0003613590651255928, 18, 18, 43998.90899658203]
-----------------
['CSCO', -0.0012675273255346301, 27, 27, -734999.4659423828]
['CSCO', -0.0015133099271022932, 19, 19, -925999.0692138672]
-----------------
['UAL', -0.006415208945305793, 37, 37, -2520999.526977539]
['UAL', -8.889098530360704e-05, 17, 17, -204001.42669677734]
-----------------
['NVR', -0.004058244135770442, 36, 36, -181